In [1]:
import pandas as pd

import requests  

import re
import math
import random
import time

from pymongo import MongoClient

In [2]:
# 设置请求头
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Host': 'www.woshipm.com',
    'Referer': 'http://www.woshipm.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
}

In [3]:
# 创建连接
client = MongoClient('localhost', 27017)
# 获取数据库
db = client['iSpider']
# 获取集合
stream = db.woshipm_stream_list
article = db.woshipm_article

In [4]:
# 获取 JSON 函数
def getJSON(url, headers):
    """ Get JSON
    @ param url: str 
    @ param headers: dict, request headers
    @ return json: JSON, result
    """
    res = requests.get(url, headers=headers) 
    res.raise_for_status()  #抛出异常
    res.encoding = 'utf-8'  
    json = res.json()
    return json

In [5]:
# 获取文章列表函数
def getArticleList(page, collection, headers):
    """ Get Article List
    @ param page: int
    @ param headers: dict, request headers
    @ param collection: pymongo.collection.Collection
    """
    url = 'http://www.woshipm.com/__api/v1/stream-list/page/{0}'.format(page)
    json = getJSON(url, headers)
    
    if json['success'] == 'true':
        for article in json['payload']:
            collection.insert_one(article)
    else:
        print(json['success'])
        
    print("获取第 {0} 页成功！".format(page))

In [9]:
for i in range(4001, 4096):
    #getArticleList(i, stream, headers)
    time.sleep(random.randint(2, 4))

获取第 4001 页成功！
获取第 4002 页成功！
获取第 4003 页成功！
获取第 4004 页成功！
获取第 4005 页成功！
获取第 4006 页成功！
获取第 4007 页成功！
获取第 4008 页成功！
获取第 4009 页成功！
获取第 4010 页成功！
获取第 4011 页成功！
获取第 4012 页成功！
获取第 4013 页成功！
获取第 4014 页成功！
获取第 4015 页成功！
获取第 4016 页成功！
获取第 4017 页成功！
获取第 4018 页成功！
获取第 4019 页成功！
获取第 4020 页成功！
获取第 4021 页成功！
获取第 4022 页成功！
获取第 4023 页成功！
获取第 4024 页成功！
获取第 4025 页成功！
获取第 4026 页成功！
获取第 4027 页成功！
获取第 4028 页成功！
获取第 4029 页成功！
获取第 4030 页成功！
获取第 4031 页成功！
获取第 4032 页成功！
获取第 4033 页成功！
获取第 4034 页成功！
获取第 4035 页成功！
获取第 4036 页成功！
获取第 4037 页成功！
获取第 4038 页成功！
获取第 4039 页成功！
获取第 4040 页成功！
获取第 4041 页成功！
获取第 4042 页成功！
获取第 4043 页成功！
获取第 4044 页成功！
获取第 4045 页成功！
获取第 4046 页成功！
获取第 4047 页成功！
获取第 4048 页成功！
获取第 4049 页成功！
获取第 4050 页成功！
获取第 4051 页成功！
获取第 4052 页成功！
获取第 4053 页成功！
获取第 4054 页成功！
获取第 4055 页成功！
获取第 4056 页成功！
获取第 4057 页成功！
获取第 4058 页成功！
获取第 4059 页成功！
获取第 4060 页成功！
获取第 4061 页成功！
获取第 4062 页成功！
获取第 4063 页成功！
获取第 4064 页成功！
获取第 4065 页成功！
获取第 4066 页成功！
获取第 4067 页成功！
获取第 4068 页成功！
获取第 4069 页成功！
获取第 4070 页成功！
获取第 4071 页成功！
获取第 40